In [1]:
import numpy as np
import cv2
import os
import random

# Algorithm

### Encryption

In [3]:
def key_generator(n_bits):
    secret_key = ''
    for i in range(n_bits):
        secret_key += chr(random.randint(33, 122))
    return secret_key

In [4]:
def ascii_and_prepend_zeros(character):
    binary = bin(ord(character))[2 : ]
    for i in range(8 - len(binary)):
        binary = '0' + binary
    return binary

In [5]:
def preprocess(image_path, size):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, size)
    return image

In [6]:
def show(image):
    cv2.imshow('Image', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [7]:
def shift_image_encrypt(image, bit_location):
    image[bit_location] = np.left_shift(np.right_shift(image[bit_location], 1), 1)
    return image

In [8]:
def bitwise_or(image, key_bit, location):
    image[location] = np.bitwise_or(image[location], int(key_bit, 2))
    return image

In [9]:
def shift_and_embed(image, key_bit, bit_location):
    image = shift_image_encrypt(image, bit_location)
    image = bitwise_or(image, key_bit, bit_location)
    return image

In [14]:
def customized_steganography(image, secret_key, binaries):
    n_locations = len(binaries) * 8
    binary = []
    for b in binaries:
        for j in range(8):
            binary.append(b[j])
    image = image.flatten()
    bit_locations = []
    for i in range(n_locations):
        location = random.randint(0, len(image) - 1)
        while location in bit_locations:
            location = random.randint(0, len(image) - 1)
        bit_locations.append(location)
        image = shift_and_embed(image, binary[i], bit_locations[i])
    return image, bit_locations

In [15]:
def encrypt(image, secret_key, size):
    binaries = []
    for character in secret_key:
        binaries.append(ascii_and_prepend_zeros(character))
    steg_image, bit_locations = customized_steganography(image, secret_key, binaries)
    return np.array(steg_image).reshape(size), bit_locations

In [16]:
size = (500, 500)
image = preprocess('image.jpeg', (size[1], size[0]))
secret_key = key_generator(2048)
print(len(secret_key))

2048


In [17]:
steg_image, bit_locations = encrypt(image, secret_key, size)

In [19]:
print(bit_locations[0 : 10])

[188398, 128789, 80615, 62222, 64823, 102706, 75901, 176204, 205656, 146170]


In [20]:
show(steg_image)

### Decryption 

In [21]:
def binarize_key(steg_image, bit_locations):
    key = []
    for i in range(len(bit_locations)):
        key.append(np.binary_repr(steg_image[bit_locations[i]], 8))
    return key

In [22]:
def image_decrypt(binarized_key):
    extracted_key = [int(str(b[7]), 10) for b in binarized_key]
    return extracted_key

In [23]:
def decrypt(steg_image, bit_locations):
    steg_image = steg_image.flatten()
    binarized_key = binarize_key(steg_image, bit_locations)
    secret_key = image_decrypt(binarized_key)
    secret_key = np.array(secret_key).reshape(len(secret_key) // 8, 8)
    key = ''
    for i in range(len(secret_key)):
        k = ''
        for j in range(8):
            k += str(secret_key[i][j])
        key += chr(int(k, 2))
    return key

In [24]:
decrypted_key = decrypt(steg_image, bit_locations)

In [25]:
secret_key == decrypted_key

True

In [26]:
secret_key

'\'0Enu7;zT`z>nUD]Cb#e7V6cn]@XYGmgB=_x)-C:v`l3jSNUQF-vra^UtRNA=eB;.%S#(jo=0.sL-&rTmO5:mobPa(#0x$RFmy$>8\'&J<H::V5Lc59imm;&7<X_\'\\kei;-[%]f))6([8<?+088:ZBHMyrS\\\'Ya"cJht>sfz,T`hyqR-NC2Z:EDkw3/o2"(`*l+i5$fO`KE:N"0"8\'Svocuaerq<;t@,\'Xy1?GOGHhb4C*bK;-UDMhMX\'gRaGn2kDbXq9^)<*YyJMq\\>@4CDZC\\&"M_HF"tJ1fsSQ\'MS6_uKEE/iU+jK2+^2hhB\\3&o@=j7c:7Fb%G_x-8Mi#"zQskfj!>FA.[,vw;\\C.hNe<Iug"*E=k5GCMx4+2j:*L[_Y=RV/%a9?o@/.fNSaItXfD"sXTms*N(#C[Jcf2pE<-Z3M^<RWgv5VFD4cgh:FLP_u?Gt,I>R5FN&g1Yt@h-s.V?.:y@7+ION8o$*uMF!p5rD?0xNh3=E>Q3(UL&PJJ?byTI7TW&`93HuqTXn2s0>5+GHj(%_eFQtet*?`4bp\';\\&-gq">N17xj&fP]Jg\\GI2ib1>sDlfuJH%GYB:O%jZ\\16W7\'P6T[<\':8hmQ_:^,*I7c\\NJH]o"g!YjvE5^0f&Fi@H(e`o*:X\\_Ufg,dngquu07QyBqdCl(gsy@sr[6gK9?zD]J^azIuY)p?,cHDZ]v1s%IiE<,I026=ey7*:s1+vA/)W$x&oS\\:JWCeRmv_L<&Z!tGpDl>r9mDJj_>eR0uO"M>83Qv%jpU>AL=a>R\'uL:>iFJtjO[MdS,wT`rpPqU4.:/+.<vkCp#`\\K`wH^+0WX!Jul"NZ]5$d6+qPYy>(+D:zX>%n/6x`v:b3_J15#W$N\\@<G0Jr2kk6YJ!&hxD@dL58okjH0a`;b)\\+sa4e3gD(>F]/-Y]W8hD\'\\RsX`,l&C$FwI@kp5Jq`?Q&m+,nFr+b"@f>iWI)b

In [27]:
decrypted_key

'\'0Enu7;zT`z>nUD]Cb#e7V6cn]@XYGmgB=_x)-C:v`l3jSNUQF-vra^UtRNA=eB;.%S#(jo=0.sL-&rTmO5:mobPa(#0x$RFmy$>8\'&J<H::V5Lc59imm;&7<X_\'\\kei;-[%]f))6([8<?+088:ZBHMyrS\\\'Ya"cJht>sfz,T`hyqR-NC2Z:EDkw3/o2"(`*l+i5$fO`KE:N"0"8\'Svocuaerq<;t@,\'Xy1?GOGHhb4C*bK;-UDMhMX\'gRaGn2kDbXq9^)<*YyJMq\\>@4CDZC\\&"M_HF"tJ1fsSQ\'MS6_uKEE/iU+jK2+^2hhB\\3&o@=j7c:7Fb%G_x-8Mi#"zQskfj!>FA.[,vw;\\C.hNe<Iug"*E=k5GCMx4+2j:*L[_Y=RV/%a9?o@/.fNSaItXfD"sXTms*N(#C[Jcf2pE<-Z3M^<RWgv5VFD4cgh:FLP_u?Gt,I>R5FN&g1Yt@h-s.V?.:y@7+ION8o$*uMF!p5rD?0xNh3=E>Q3(UL&PJJ?byTI7TW&`93HuqTXn2s0>5+GHj(%_eFQtet*?`4bp\';\\&-gq">N17xj&fP]Jg\\GI2ib1>sDlfuJH%GYB:O%jZ\\16W7\'P6T[<\':8hmQ_:^,*I7c\\NJH]o"g!YjvE5^0f&Fi@H(e`o*:X\\_Ufg,dngquu07QyBqdCl(gsy@sr[6gK9?zD]J^azIuY)p?,cHDZ]v1s%IiE<,I026=ey7*:s1+vA/)W$x&oS\\:JWCeRmv_L<&Z!tGpDl>r9mDJj_>eR0uO"M>83Qv%jpU>AL=a>R\'uL:>iFJtjO[MdS,wT`rpPqU4.:/+.<vkCp#`\\K`wH^+0WX!Jul"NZ]5$d6+qPYy>(+D:zX>%n/6x`v:b3_J15#W$N\\@<G0Jr2kk6YJ!&hxD@dL58okjH0a`;b)\\+sa4e3gD(>F]/-Y]W8hD\'\\RsX`,l&C$FwI@kp5Jq`?Q&m+,nFr+b"@f>iWI)b

# Attack

In [28]:
def image_decrypt_attack(binarized_key, position):
    extracted_key = [int(str(b[position]), 10) for b in binarized_key]
    return extracted_key

In [34]:
def attack(steg_image, bit_locations, position):
    steg_image = steg_image.flatten()
    binarized_key = binarize_key(steg_image, bit_locations)
    secret_key = image_decrypt_attack(binarized_key, position)
    secret_key = np.array(secret_key).reshape(len(secret_key) // 8, 8)
    key = ''
    for i in range(len(secret_key)):
        k = ''
        for j in range(8):
            k += str(secret_key[i][j])
        key += chr(int(k, 2))
    return key

In [42]:
position = 0
recovered_key = ''
while ((position < 8) and (recovered_key != secret_key)):
    recovered_key = attack(steg_image, bit_locations, position)
    position += 1
    
if (recovered_key == secret_key):
    print(f"Attack successful after checking {position} different bit positions")
else:
    print("Attack not successful")

Attack successful after checking 8 different bit positions
